In [ ]:
import openai
import numpy as np
from lime.lime_text import LimeTextExplainer
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Define prediction function
def gpt_sentiment_predict(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Classify this text as Positive or Negative only: '{text}'"}
        ],
        temperature=0
    )
    return response["choices"][0]["message"]["content"].strip()

In [ ]:
# Probability-like wrapper for LIME
def gpt_predict_proba(texts):
    preds = []
    for text in texts:
        label = gpt_sentiment_predict(text)
        if "Positive" in label:
            preds.append([0.1, 0.9])
        elif "Negative" in label:
            preds.append([0.9, 0.1])
        else:
            preds.append([0.5, 0.5])
    return np.array(preds)

In [ ]:
# Load and sample data
base_dir = os.getcwd()
print("Current Directory:", base_dir)
file_path = os.path.join(base_dir, "new_channel_partner_feedback.csv")
df = pd.read_csv(file_path)
sample_texts = df["Feedback_Text"].dropna().sample(4, random_state=42).tolist()

In [ ]:
# Initialize LIME explainer
explainer = LimeTextExplainer(class_names=["Negative", "Positive"])

In [ ]:
# Explain each sample
for i, text in enumerate(sample_texts):
    print(f"\n--- Explanation for sample {i+1} ---\n{text}\n")
    exp = explainer.explain_instance(text, gpt_predict_proba, num_features=10, num_samples=50)
    exp.show_in_notebook()